In [50]:
import os
import threading
import time
import datetime, random
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity

from IPython.display import HTML
import plotly.express as px
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import re

In [22]:
dataloc = "data/images/"
outputloc = "output"
labels = "data/labels"
rdcl_path = "data/RDCL2019/"

In [23]:
all_files = []
files_list = os.listdir(rdcl_path)

Following goe shtouhg the example RDCL2019 dataset. this reads xml and extracts the raw text. then run the .tiff through documents kraken and tesseract

In [24]:
os.environ['MKL_SERVICE_FORCE_INTEL'] = '1'

textfiles = {}
location = []
for file in files_list:
    input_file = "{}{}".format(rdcl_path, file)

    output_file_text = "{}/RDCL2019/text/{}.txt".format(outputloc, file[:-4])
    
    #we get unicode out of the xml file
    
    if file[-3:] != 'xml' and file[-3:] != 'tif':
        continue
    
    if file[-3:] == 'xml':
        
        fd = open(input_file, 'r')
        
        xml_file = fd.read()

        soup = BeautifulSoup(xml_file, 'lxml')
        
        od = open(output_file_text, 'w')

        for tag in soup.findAll('unicode'):
            od.write(tag.text)
            #print(tag.text)
        od.close()
        
        continue
    
    #run the none xml files though kraken and tesseract for later. 
    output_file_kraken = "{}/RDCL2019/kraken/{}.txt".format(outputloc, file[:-4])
    output_file_tesseract = "{}/RDCL2019/tesseract/{}".format(outputloc, file[:-4])
        
    system_cmd_k = "kraken -i {} {} binarize segment ocr -m en_best.mlmodel".format(input_file, output_file_kraken)
    system_cmd_t = "tesseract {} {}".format(input_file, output_file_tesseract)
    
    pro_time = time.process_time()
    os.system(system_cmd_k)
    location.append(["kraken", time.process_time() - pro_time, input_file, output_file_kraken, output_file_text ])
    
    
    pro_time = time.process_time()
    os.system(system_cmd_t)
    location.append(["tesseract", time.process_time() - pro_time, input_file, "{}.txt".format(output_file_tesseract), output_file_text ])
    

In [38]:
for loc in location:
    print(loc)

['kraken', 0.0003861160000000474, 'data/RDCL2019/00770911.tif', 'output/RDCL2019/kraken/00770911.txt', 'output/RDCL2019/text/00770911.txt', ["me'3-=- y rinthebagmind and body stimulation1  - i]koregin 300o0 is a traditionalenergy booster used to help the]body cope with physical andpsychological stress and] infections.", 'koregin 3000]capsules contain highly: iiroot extract.', 'they provide 600mg of the finest quality31iin [estessee]from herbs grown in thekorean foothills.', 'koreani1]aciiioalit promotes an overall feelingu]ofwellbeing.', 'research has also lmay also help men who suffertfrom impotence, by improvingerectile function.', 'koregin]3000 retails at e10.99 for30 capsules.', ']contact healthaid on]r4420 8426 340o for purchaseand stockist information,or visit www.healthaid.co.uk::.', 'hayfeverthese handsome and elegant designs w ill] new organic haymaxvocontinue to hold their own and safely hold aloe vera combines the known-eko:e..:.', ';i1 finest black or brown smooth calf leat

We want statistics BLEU, Rouge n-rouge 

In [74]:
import warnings
warnings.filterwarnings('ignore')

import nltk.translate.bleu_score as bleu 
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from gensim.models.doc2vec import Doc2Vec, TaggedDocument 

In [81]:
hold = open(location[0][4]).read()
article = re.sub("\n","", hold.lower())
print(article)

doc = CountVectorizer(hold)

print(doc)

advertisement featurestay on trendmind and body stimulationhayfeverin the bagintroducing a new range of messengerbags from aspinal of london. as seasonscome and go and trends emerge and fade,these handsome and elegant designs willcontinue to hold their own and safely holdall your essentials. aspinal messengers andstylish satchels are made from the veryfinest black or brown smooth calf leatherand are stain and water treated for extradurability. there are also cool canvas andcompact versions available. the fullyadjustable leather or contrast canvasstraps are individually hand-stitched withbrass or nickel ﬁttings. inside, there arepockets to hold your documents, pens andphone and the larger styles are sturdy andcapacious enough to carry a lightweightlaptop. sophisticated and sleek, theaspinal range of messengers effortlesslymerges classic craftsmanship with sharp,contemporary styling. aspinal carriesa wide range of handmade leatheraccessories and bags including laptop andweekend bags, bri

In [61]:

for loc in location:
    tfidf_vectorizer = TfidfVectorizer(min_df=1, stop_words="english")
    
    ocr_file = open(loc[3])
    ocr_file_string = ocr_file.read()
    ocr_tok = sent_tokenize(re.sub("\n","", ocr_file_string.lower()))
    
    a = np.array(ocr_tok)
    
    print(a)
    
    t_ocr_file = open(loc[4])
    t_ocr_file_string = t_ocr_file.read()
    t_ocr_txt =  sent_tokenize(re.sub("\n","", t_ocr_file_string.lower()))
    
    print(t_ocr_txt)
    
    

["me'3-=- y rinthebagmind and body stimulation1  - i]koregin 300o0 is a traditionalenergy booster used to help the]body cope with physical andpsychological stress and] infections."
 'koregin 3000]capsules contain highly: iiroot extract.'
 'they provide 600mg of the finest quality31iin [estessee]from herbs grown in thekorean foothills.'
 'koreani1]aciiioalit promotes an overall feelingu]ofwellbeing.'
 'research has also lmay also help men who suffertfrom impotence, by improvingerectile function.'
 'koregin]3000 retails at e10.99 for30 capsules.'
 ']contact healthaid on]r4420 8426 340o for purchaseand stockist information,or visit www.healthaid.co.uk::.'
 'hayfeverthese handsome and elegant designs w ill] new organic haymaxvocontinue to hold their own and safely hold aloe vera combines the known-eko:e..:.'
 ';i1 finest black or brown smooth calf leather] soothing and skin repair effectsand are stain and water treated for extra] of aloe vera.durability.'
 'there are also cool canvas and  

In [25]:
ocr_file = open(location[0][3])
ocr_tok = word_tokenize(ocr_file.read())
print(ocr_tok)

FileNotFoundError: [Errno 2] No such file or directory: 'output/RDCL2019/kraken/00848894-pc-gt.txt'

In [27]:
location[0][3]

'output/RDCL2019/kraken/00848894-pc-gt.txt'